In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd 
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score,roc_auc_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import math
from sklearn.naive_bayes import BernoulliNB 
from sklearn.base import BaseEstimator,ClassifierMixin
from sklearn.model_selection import GridSearchCV
import pickle 
import numpy as np
from collections import Counter
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier


In [2]:
def entropy(y):
    hist = np.bincount(y)
    ps = hist / len(y)
    return -np.sum([p * np.log2(p) for p in ps if p > 0])

def bootstrap_sample(X, y):
    n_samples = X.shape[0]
    idxs = np.random.choice(n_samples, n_samples, replace=True)
    return X[idxs], y[idxs]

def most_common_label(y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common

def metrics(model):
    
    print(f'Доля верных ответов на обучающей выборки = {accuracy_score(y_train, model.predict(X_train))}')
    print(f'Доля верных ответов на тестовой выборки = {accuracy_score(y_test, model.predict(X_test))}')
    
    print(f'Recall_score на обучающей выборки = {recall_score(y_train, model.predict(X_train))}')
    print(f'Recall_score на тестовой выборки = {recall_score(y_test, model.predict(X_test))}')
    
    print(f'Precision_score на обучающей выборки = {precision_score(y_train, model.predict(X_train))}')
    print(f'Precision_score на тестовой выборки = {precision_score(y_test, model.predict(X_test))}')
    
    print(f'Roc_auc_score на обучающей выборки = {roc_auc_score(y_train, model.predict(X_train))}')
    print(f'Roc_auc_score на тестовой выборки = {roc_auc_score(y_test, model.predict(X_test))}')
    
    print(f'Confusion_matrix на обучающей выборки = \n{confusion_matrix(y_train, model.predict(X_train))}')
    print(f'Confusion_matrix на тестовой выборки = \n{confusion_matrix(y_test, model.predict(X_test))}')

In [3]:
dataset = pd.read_csv('remastered_dataset.csv')

In [4]:
X_train, X_test, y_train, y_test = train_test_split(np.array(dataset.drop(['Survived'],axis=1)), np.array(dataset['Survived']), test_size=0.2, random_state=42)

### Дерево решений 


In [5]:
class Node:

    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None
    
class DecisionTree:

    def __init__(self, min_samples_split=2, max_depth=50, n_feats=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None

    def fit(self, X, y):
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
        self.root = self._grow_tree(X, y)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # stopping criteria
        if (depth >= self.max_depth
                or n_labels == 1
                or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)

        # greedily select the best split according to information gain
        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)
        
        # grow the children that result from the split
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feat, best_thresh, left, right)

    def _best_criteria(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_thresh = None, None
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = threshold

        return split_idx, split_thresh

    def _information_gain(self, y, X_column, split_thresh):
        # parent loss
        parent_entropy = entropy(y)

        # generate split
        left_idxs, right_idxs = self._split(X_column, split_thresh)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # compute the weighted avg. of the loss for the children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = entropy(y[left_idxs]), entropy(y[right_idxs])
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r

        # information gain is difference in loss before vs. after split
        ig = parent_entropy - child_entropy
        return ig

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def _most_common_label(self, y):
        if len(y) == 0:
            return 1
        else:
            
            counter = Counter(y)
            
            most_common = counter.most_common(1)[0][0]
            return most_common

In [6]:
model = DecisionTree()
model.fit(X_train, y_train)
metrics(model)

Доля верных ответов на обучающей выборки = 0.9845505617977528
Доля верных ответов на тестовой выборки = 0.776536312849162
Recall_score на обучающей выборки = 0.9776119402985075
Recall_score на тестовой выборки = 0.7702702702702703
Precision_score на обучающей выборки = 0.9812734082397003
Precision_score на тестовой выборки = 0.7125
Roc_auc_score на обучающей выборки = 0.9831753395186231
Roc_auc_score на тестовой выборки = 0.7756113256113256
Confusion_matrix на обучающей выборки = 
[[439   5]
 [  6 262]]
Confusion_matrix на тестовой выборки = 
[[82 23]
 [17 57]]


In [7]:
model = DecisionTreeClassifier()
model.fit(X_train,y_train)
metrics(model)

Доля верных ответов на обучающей выборки = 0.9845505617977528
Доля верных ответов на тестовой выборки = 0.7932960893854749
Recall_score на обучающей выборки = 0.9626865671641791
Recall_score на тестовой выборки = 0.7432432432432432
Precision_score на обучающей выборки = 0.9961389961389961
Precision_score на тестовой выборки = 0.7534246575342466
Roc_auc_score на обучающей выборки = 0.9802171574559635
Roc_auc_score на тестовой выборки = 0.7859073359073359
Confusion_matrix на обучающей выборки = 
[[443   1]
 [ 10 258]]
Confusion_matrix на тестовой выборки = 
[[87 18]
 [19 55]]


### Random forest 

In [8]:
class RandomForest:
    
    def __init__(self, n_trees=10, min_samples_split=2,
                 max_depth=100, n_feats=None):
        self.n_trees = n_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(min_samples_split=self.min_samples_split,
                max_depth=self.max_depth, n_feats=self.n_feats)
            X_samp, y_samp = bootstrap_sample(X, y)
            tree.fit(X_samp, y_samp)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(tree_preds, 0, 1)
        y_pred = [most_common_label(tree_pred) for tree_pred in tree_preds]
        return np.array(y_pred)

In [9]:
model = RandomForest()
model.fit(X_train, y_train)
metrics(model)

Доля верных ответов на обучающей выборки = 0.9705056179775281
Доля верных ответов на тестовой выборки = 0.770949720670391
Recall_score на обучающей выборки = 0.9477611940298507
Recall_score на тестовой выборки = 0.6756756756756757
Precision_score на обучающей выборки = 0.9731800766283525
Precision_score на тестовой выборки = 0.746268656716418
Roc_auc_score на обучающей выборки = 0.9659977141320426
Roc_auc_score на тестовой выборки = 0.7568854568854568
Confusion_matrix на обучающей выборки = 
[[437   7]
 [ 14 254]]
Confusion_matrix на тестовой выборки = 
[[88 17]
 [24 50]]


In [10]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
metrics(model)

Доля верных ответов на обучающей выборки = 0.9845505617977528
Доля верных ответов на тестовой выборки = 0.8100558659217877
Recall_score на обучающей выборки = 0.9664179104477612
Recall_score на тестовой выборки = 0.7567567567567568
Precision_score на обучающей выборки = 0.9923371647509579
Precision_score на тестовой выборки = 0.7777777777777778
Roc_auc_score на обучающей выборки = 0.9809567029716284
Roc_auc_score на тестовой выборки = 0.8021879021879021
Confusion_matrix на обучающей выборки = 
[[442   2]
 [  9 259]]
Confusion_matrix на тестовой выборки = 
[[89 16]
 [18 56]]


### Gradient boosting

In [11]:
model = GradientBoostingClassifier()
model.fit(X_train, y_train)
metrics(model)

Доля верных ответов на обучающей выборки = 0.9044943820224719
Доля верных ответов на тестовой выборки = 0.8156424581005587
Recall_score на обучающей выборки = 0.7873134328358209
Recall_score на тестовой выборки = 0.7162162162162162
Precision_score на обучающей выборки = 0.9504504504504504
Precision_score на тестовой выборки = 0.8153846153846154
Roc_auc_score на обучающей выборки = 0.8812693290305231
Roc_auc_score на тестовой выборки = 0.800965250965251
Confusion_matrix на обучающей выборки = 
[[433  11]
 [ 57 211]]
Confusion_matrix на тестовой выборки = 
[[93 12]
 [21 53]]


### Мягкое и жесткое голосование 

In [12]:
def soft(models,X):
    predicts = [i.predict_proba(X)[:,1] for i in models]
    
    
    back = np.zeros(predicts[0].shape)
    for i in range(len(predicts)):
        back = back + np.array(predicts[i])
    back = back / len(predicts)
    
    
    real_back = [1 if back[i]>=0.5 else 0 for i in range(len(back))]
    
    
    return real_back


def hard(models,X):
    predicts = [i.predict_proba(X)[:,1] for i in models]
    
    back = np.zeros(predicts[0].shape)
    
    for i in range(len(predicts)):
        back = back + predicts[i]
    back = back / len(predicts)
    
    
    for i in range(len(back)):
        if back[i]>=0.5:
            back[i] = 1
        else:
            back[i] = 0
    
    return back

In [13]:
models = [pickle.load(open(i, 'rb')) for i in ['sklearn_KNN()_best_model.pkl','sklearn_LogitRegression_best_model.pkl'
                                    ,'sklearn_NaiveBayesClassifier()_best_model.pkl','sklearn_SVM()_best_model.pkl']]

In [14]:
accuracy_score(soft(models,dataset.drop(['Survived'],axis=1)),dataset['Survived'])

0.8069584736251403

In [15]:
accuracy_score(hard(models,dataset.drop(['Survived'],axis=1)),dataset['Survived'])

0.8069584736251403